# **Programming Assessment \#4**

Names: Alyanna Abalos, Loben Tipan

More information on the assessment is found in our Canvas course.

# **Load Data**

*While you don't have to separate your code into blocks, it might be easier if you separated loading your data from actually implementation of your code. Consider placing all loading of data into the code block below.*

In [1]:
with open('count_1edit.txt', 'r', encoding='ISO-8859-1') as file:
    data_count_1edit = file.read()

with open('spell_errors.txt', 'r', encoding='ISO-8859-1') as file:
    data_spell_errors = file.read()

# **Error Model Implementation**

In [2]:
from collections import defaultdict

error_model = {}
for line in data_count_1edit.splitlines():
    parts = line.split('|')
    w = parts[0].strip() 
    
    if len(parts) < 2:
        continue

    c_split = parts[1].split()
    if len(c_split) < 2:
        continue

    c = c_split[0]
    count = int(c_split[1])
    
    error_model[(w, c)] = count

total_counts_for_correct = defaultdict(int)
for (corrupted, correct), count in error_model.items():
    total_counts_for_correct[correct] += count

for (corrupted, correct) in error_model:
    error_model[(corrupted, correct)] /= total_counts_for_correct[correct]

spell_errors = {}
for line in data_spell_errors.splitlines():
    correct_word, misspelled_words_part = line.split(":")
    misspelled_words = [word.strip() for word in misspelled_words_part.split(',')]
    spell_errors[correct_word.strip()] = misspelled_words

print(list(error_model.items())[:5], "\n\n")
print(list(spell_errors.items())[:5])

[(('e', 'i'), 0.3411458333333333), (('a', 'e'), 0.24160316116285635), (('i', 'e'), 0.21761219305673157), (('e', 'a'), 0.2843583902809415), (('a', 'i'), 0.20796130952380953)] 


[('raining', ['rainning', 'raning']), ('writings', ['writtings']), ('disparagingly', ['disparingly']), ('yellow', ['yello']), ('four', ['forer', 'fours', 'fuore', 'fore*5', 'for*4'])]


# **Language Model Implementation**

In [3]:
import nltk
nltk.download('gutenberg')

from nltk.corpus import gutenberg
from collections import Counter

def build_language_model():
    words = gutenberg.words()
    words = [word.lower() for word in words if word.isalpha()]
    word_freq = Counter(words)
    total_words = sum(word_freq.values())
    language_model = {word: freq / total_words for word, freq in word_freq.items()}
    return language_model

language_model = build_language_model()
vocabulary = set(language_model.keys())

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\loben\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


# **Candidate Generation**

In [4]:
def generate_candidates(word, vocabulary):
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    candidates = []
    
    for i in range(len(word)):
        candidate = word[:i] + word[i+1:]
        if candidate in vocabulary:
            candidates.append((candidate, "del"))
    
    for i in range(len(word) + 1):
        for letter in alphabet:
            candidate = word[:i] + letter + word[i:]
            if candidate in vocabulary:
                candidates.append((candidate, "ins"))
    
    for i in range(len(word)):
        for letter in alphabet:
            if word[i] != letter:  # Ensure we're changing the character
                candidate = word[:i] + letter + word[i+1:]
                if candidate in vocabulary:
                    candidates.append((candidate, "sub"))
    
    for i in range(len(word) - 1):
        candidate = word[:i] + word[i+1] + word[i] + word[i+2:]
        if candidate in vocabulary:
            candidates.append((candidate, "tra"))
    
    return candidates


word = "mohter"
candidates = generate_candidates(word, vocabulary)

# **Edit Identifier**

In [5]:
def get_character_edit(word, candidate, operation):
    """Return the character edit operation between word and candidate."""
    if operation == "sub":
        for i in range(len(word)):
            if word[i] != candidate[i]:
                return "{}|{}".format(word[i], candidate[i])
    elif operation == "ins":
        for i in range(len(word)):
            if i == len(candidate) or word[i] != candidate[i]:
                return f"|{candidate[i]}"
        return f"|{candidate[-1]}"
    elif operation == "del":
        for i in range(len(candidate)):
            if i == len(word) or word[i] != candidate[i]:
                return f"{word[i]}|"
        return f"{word[-1]}|"
    elif operation == "tra":
        for i in range(len(word) - 1):
            if (word[i] != candidate[i]) and (word[i] == candidate[i+1]) and (word[i+1] == candidate[i]):
                return "{}{}|{}{}".format(word[i], word[i+1], candidate[i], candidate[i+1])
    return "N/A"

# **Probabilities**

In [6]:
import math

def compute_correction_probabilities_log(word, candidates, error_model, language_model):
    probabilities = []
    SMOOTHING_FACTOR = 0.000001  # Define a small smoothing factor

    for candidate, op in candidates:
        # Determine character edit
        char_edit = get_character_edit(word, candidate, op)
        
        # Get log error probability log P(c|w) with smoothing for unseen edits
        log_p_c_given_w = math.log(error_model.get((char_edit), SMOOTHING_FACTOR))
        
        # Get log language model probability log P(w) with default value of log(0) if not found
        log_p_w = math.log(language_model.get(candidate, SMOOTHING_FACTOR))
        
        # Compute combined log probability log P(w|c) = log P(c|w) + log P(w)
        log_p_w_given_c = log_p_c_given_w + log_p_w
        probabilities.append((candidate, op, log_p_w, log_p_c_given_w, log_p_w_given_c))
    
    # Sort by combined log probability
    probabilities.sort(key=lambda x: x[-1], reverse=True)
    
    return probabilities



probabilities = compute_correction_probabilities_log(word, candidates, error_model, language_model)


In [8]:
if word in vocabulary:
    print(f"'{word}' is already correctly spelled.")
else:
    header = "{:<15} {:<15} {:<10} {:<20} {:<15} {:<15} {:<15}"
    print(header.format("word", "candidate", "edit_type", "edit", "log P(c)", "log P(w|c)", "P(c) x P(w|c)"))
    for row in probabilities:
        candidate, op, log_p_w, log_p_c_given_w, log_p_w_given_c = row
        char_edit_str = get_character_edit(word, candidate, op)
        # Convert log probabilities back to normal scale for display
        p_w = math.exp(log_p_w)
        p_c_given_w = math.exp(log_p_c_given_w)
        p_w_given_c = math.exp(log_p_w_given_c)
        # Display more decimal places to avoid rounding to zero
        print(header.format(word, candidate, op, char_edit_str, f"{log_p_w:.6f}", f"{log_p_c_given_w:.6f}", f"{p_w_given_c:.10f}"))



IndentationError: expected an indented block (2568843200.py, line 7)